### This code is run twice, once to create the all samples db and once for the lite db. It reads the .parquest files and converts them to MySQL tables in the required database.

In [37]:
import os
import pymysql
import pandas as pd

with open('pass.txt', 'r') as file:
    password = file.read()

# database= "SEMEVALSAMPLE" OR "SEMEVALALL"

mysql_config = {
    "host": "localhost",  # MySQL host
    "user": "root",  # MySQL username
    "password": password,  # MySQL password
    "database": "SEMEVALSAMPLE"  # MySQL database name
}

# directory containing the folders
base_dir = "/Users/akshitsharma/Downloads/competition/"  # directory path

# Connect to MySQL
connection = pymysql.connect(**mysql_config)
cursor = connection.cursor()

# Create the database if it doesn't exist
db_name = mysql_config["database"]
# cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name}")
cursor.execute(f"USE {db_name}")

dtype_map = {
    'uint8': 'INT',
    'uint16': 'INT',
    'uint32': 'INT',
    'float64': 'DOUBLE',
    'object': 'TEXT',
    'bool': 'TINYINT(1)',
    'datetime64[ns, UTC]': 'DATETIME',
    'category': "TEXT",
}


# Traverse through the directories
for folder_name in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder_name)

    if os.path.isdir(folder_path):
        file_path = os.path.join(folder_path, "sample.parquet")
        
        # Check if the all.parquet/sample.parquet file exists in the folder
        if os.path.exists(file_path):
            df = pd.read_parquet(file_path)
            table_name = folder_name
            
            columns = []
            for col, dtype in df.dtypes.items():
                mysql_type = dtype_map.get(str(dtype), 'TEXT')  # Default to TEXT if dtype not mapped
                columns.append(f"`{col}` {mysql_type}")
                
            create_table_query = f"CREATE TABLE IF NOT EXISTS `{table_name}` ({','.join(columns)})"
            print(create_table_query)
            cursor.execute(create_table_query)
            
            # Insert data into the table
            for _, row in df.iterrows():
                # Convert NaN to None
                row = tuple(None if pd.isna(val) else val for val in row)
                placeholders = ", ".join(["%s"] * len(row))
                insert_query = f"INSERT INTO `{table_name}` VALUES ({placeholders})"
                cursor.execute(insert_query, row)
            
            print(f"Created table '{table_name}' for file: {file_path}")
        else:
            print(f"No 'all.parquet' file found in folder: {folder_name}")

# Commit changes and close the connection
connection.commit()
cursor.close()
connection.close()

CREATE TABLE IF NOT EXISTS `071_COL` (`Rank` INT,`Country` TEXT,`Cost of Living Index` DOUBLE,`Rent Index` DOUBLE,`Cost of Living Plus Rent Index` DOUBLE,`Groceries Index` DOUBLE,`Restaurant Price Index` DOUBLE,`Local Purchasing Power Index` DOUBLE)
Created table '071_COL' for file: /Users/akshitsharma/Downloads/competition/071_COL/sample.parquet
CREATE TABLE IF NOT EXISTS `077_Gestational` (`Age` INT,`Pregnancy No` INT,`Weight` INT,`Height` INT,`BMI` DOUBLE,`Heredity` INT,`Prediction` INT)
Created table '077_Gestational' for file: /Users/akshitsharma/Downloads/competition/077_Gestational/sample.parquet
CREATE TABLE IF NOT EXISTS `075_Mortality` (`rownames` INT,`Region` TEXT,`Status` TEXT,`Sex` TEXT,`Cause` TEXT,`Rate` DOUBLE,`SE` DOUBLE)
Created table '075_Mortality' for file: /Users/akshitsharma/Downloads/competition/075_Mortality/sample.parquet
CREATE TABLE IF NOT EXISTS `073_Med_Cost` (`age` INT,`sex` TEXT,`bmi` DOUBLE,`children` INT,`smoker` TEXT,`region` TEXT,`charges` DOUBLE)
Cr